## ChatLab: Chat Experiments, Simplified

💬🔬

In [1]:
import chatlab

chat = chatlab.Chat()

await chat("What kind of bird is a murkrow?")

Murkrow is a fictional Pokémon species in the Pokémon franchise. It is a Dark/Flying type Pokémon that resembles a crow or raven.

In [2]:
from chatlab import Chat, user, system

chat = Chat()

await chat(
    system("You are a data science tutor running inside of a Jupyter "
           "Notebook environment that has the full NumFOCUS/PyData stack."),
    user("Help me work with circulation-grade-6.csv"),
)

Sure! To start working with the "circulation-grade-6.csv" file, we need to load it into a pandas DataFrame. Here's how you can do it:

```python
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('circulation-grade-6.csv')

# Display the first few rows of the DataFrame
df.head()
```

This code will load the CSV file into a DataFrame called `df` and display the first few rows of the DataFrame. You can adjust the number of rows to display by passing an argument to the `head()` function.

Once you have loaded the data, you can perform various operations on it, such as data cleaning, exploration, and analysis. Let me know what specific tasks or analysis you would like to perform on the data, and I'll be happy to assist you further.

## Introducing Chat Functions

Where `chatlab` (and OpenAI, really) take it next level is with _Chat Functions_. You can declare a function with a schema then register it in your `Chat` for GPT Models to call. You may recall this kind of behavior from [ChatGPT Plugins](https://noteable.io/chatgpt-plugin-for-notebook/). Now, you can take this even further with your own custom code.

As an example, let's give the large language models the ability to tell time.

In [3]:
from datetime import datetime
from pytz import timezone, all_timezones, utc
from typing import Optional
from pydantic import BaseModel

def what_time(tz: Optional[str] = None):
    '''Current time, defaulting to UTC'''
    if tz is None:
        pass
    elif tz in all_timezones:
        tz = timezone(tz)
    else:
        return 'Invalid timezone'
        
    return datetime.now(tz).strftime('%I:%M %p')

class WhatTime(BaseModel):
    tz: Optional[str]

Let's break this down.

`what_time` is the function we're going to provide access to. Its docstring forms the `description` for the model while the schema comes from the pydantic `BaseModel` called `WhatTime`.

In [4]:
import chatlab

chat = chatlab.Chat()

# Register our function
chat.register(what_time, WhatTime)

FunctionDefinition(name='what_time', description='Current time, defaulting to UTC', parameters={'properties': {'tz': {'anyOf': [{'type': 'string'}, {'type': 'null'}]}}, 'required': ['tz'], 'type': 'object'})

In [5]:
await chat("What time is it?")

<div><style>.chatlab-chat-details summary &gt; *  { display: inline; color: #27374D; }</style><details style="background: #DDE6ED; border-radius: 5px; padding: .5rem 1rem" className="chatlab-chat-details"><summary style="color: #27374D; cursor: pointer"><span style="color: #9DB2BF; padding-left: 5px; padding-right: 5px">𝑓</span><span style="color: #27374D; padding-left: 5px; padding-right: 5px">Ran</span><span style="font-family: monospace; unicode-bidi: embed; white-space: pre">what_time</span><span style="font-family: monospace; unicode-bidi: embed; white-space: pre">...</span></summary><div style="margin-left: 10px; margin-top: 10px"><div><div style="color: #27374D; font-weight: 500; margin-bottom: 5px">Input:</div><div style="background: #F7F9FA; color: #27374D; font-family: monospace; margin-bottom: 10px; overflow-x: auto; padding: 10px; unicode-bidi: embed; white-space: pre">{
  &quot;tz&quot;: null
}</div></div><div></div></div></details></div>

The current time is 12:44 PM.

In [6]:
chat.messages

[{'role': 'user', 'content': 'What time is it?'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_YNqXo5IGQHFNkHXbacF1cfM7',
    'function': {'name': 'what_time', 'arguments': '{\n  "tz": null\n}'},
    'type': 'function'}]},
 {'role': 'tool',
  'name': 'what_time',
  'content': '12:44 PM',
  'tool_call_id': 'call_YNqXo5IGQHFNkHXbacF1cfM7'},
 {'role': 'assistant', 'content': 'The current time is 12:44 PM.'}]

In [7]:
await chat("Great. What time is it in Osaka?")

The current time in Osaka is 10:08 AM.

Under the hood, `chatlab` is keeping track of the conversation history for you in `.messages`

In [8]:
chat.messages

[{'role': 'user', 'content': 'What time is it?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time', 'arguments': '{}'}},
 {'role': 'function', 'content': '06:08 PM', 'name': 'what_time'},
 {'role': 'assistant', 'content': ''},
 {'role': 'user', 'content': 'Great. What time is it in Osaka?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'what_time',
   'arguments': '{\n  "tz": "Asia/Tokyo"\n}'}},
 {'role': 'function', 'content': '10:08 AM', 'name': 'what_time'},
 {'role': 'assistant', 'content': ''}]

## Outro

Thanks for reading through! Please post issues and reach out. I want to hear your experiences!